# Lab Immersion Notebook
#### *Emilio Fernández Lavado, EPFL*
This notebook describes the different tasks carried out as part of my immersion in the Behavioural Genetics Lab of EPFL (Prof. Carmen Sandi). The main goal during my lab immersion was to make use of my coding skills to help PhD candidate Jun Huang with his research project for understanding the underlying neural mechanisms of reward-based learning with detailed analysis of neural and behavioural data. The contents of this notebook are separated in the following subparts for easing the reader experience:
* **[Project introduction](#intro)** - brief desciption of the project scope and general aims
* **[Computer vision models](#vision)** - outline of the methods used for tracking animal behaviour 
* **[Animal behaviour analysis](#deeplab)** - discussion of resuts obtained after DeepLabCut pose estimation
* **[Neural data clustering](#cluster)** - exploratory inspection of experimental neural data for pattern identification


<a id = 'intro'></a>
## 1. Project introduction
The research project focuses on studying the **dopamine release** in NAc (nucleus accumbens) and DLS (dorsal striatum) during **reward-based learning** in order to identify key neural mechanisims linking accurate neural data with animal behavioural patterns. The dopamine relase is measured in real time with **fiber photometry** of flourescence signals from the dopamine sensor dLight expressed in the nucleus accumbens of mice. The task of the mice consisted on licking a glucose liquid from a spout whenever after **whisker stimulation**. The learning process was composed on several phases to ensure the correct habituation of the mice to the environment as well as the different cases in terms of lick/reward relationship. 

<div align="middle">
  <img width="350" height="400" src="./Multimedia/neuralrecording.jpg">
  <img width="650" height="400" src="./Multimedia/behaviour_paradigm.jpg">
</div>

The fiber-photometric recording of dopamine is complemented by the **filming of the mice behaviour** during every trial to check if they are actually licking the spout as well as observing any interesting behavioural pattern that could be link to distinguising the learning rates of the different animals involved in the experiment. Therefore, the main objective of my lab immersion is to intelligently **track several facial points** of the mice using computer vision models for its later analysis and cross-validation of behavioural data with neural data.

<a id = 'vision'></a>
## 2. Computer vision models
Computer vision can be define as the automation of tasks that the human visual system can do using image processing techniques or machine learning algorithms, in order to gain high-level understanding from digital multimedia using computer computational power. In this case, the multimedia to analyse is the recording of the mice face during every trial of its learning process (free licking, whisker detection task, delayed reward task...)

### 2.1 Video compression
The videos are filmed, stored and transformed to the codec [FFV1](http://download.das-werkstatt.com/pb/mthk/info/video/comparison_video_codecs_containers.html#codec_ffv1) which is a **compressed loseless video format**. All the details about the compression can be found on the [video_compression](./video_compression.ipynb) notebook. As a result, we obtain videos with 256x256 pixels and 100 frames/second. They are organised in folders by animal, by session and by trial 

### 2.2 Spout detection
The most important aspect of our computer vision model must be its ability to detect if the mouse is actually licking. For that purpose, it is essential to estimate the spout position, so that it can be chosen as a **fixed reference point** which can be compared with the tongue position to detect when the animal is licking. Given that the spout is a separate object which is in general not overlapped with the animal face, the most straight-forward way of estimating the spout position is using **edge detection**, which consists on finding the boundaries of objects within images using **morphological operations** such as dilation, closing or threshold functions. This image processing technique is sucessfully implemented on [spout_detection](./spout_detection.ipynb) notebook and, as a result, we are able to retrieve the spout coordinates using only one frame per session. Before applying the morphological filter, a Gaussian blurring is applied for homogenizing the edges of the image. Finally, the spout is detected using the Blob detection function from OpenCV python library.

<div align="middle">
  <img width="800" height="200" src="./Multimedia/spoutdetection.jpg">
</div>

### 2.3 Object tracking with machine learning
After the spout is correctly located, the tongue and other facial points must be tracked for the whole duration of every video to detect when the mouse is licking and how it is behaving during the trial. This task is much more difficult than the spout detection because of the mouse motion. For example, the tongue is not always present on the image because the mouse is not always licking, Therefore, we should **discard edge detection** for tracking these points as well as any other image processing technique which does not offer a possibility to detect dynamic objects. Luckily for us, computer vision has already solved this problem thanks to the **use of convolutional neural networks**. Using supervised machine learning, we can train an adapted neural network for dynamically tracking the facial points that we are interested in. Moreover, it could also track the spout with the advantage of providing a much more general and abstract technique for the spout detection instead of having to tune the morphological filters for different light or contrast conditions. The implementation and results of this neural network are discussed in the next section.



<a id = 'deeplab'></a>
## 3. Animal behaviour analysis

### 3.1 Tongue tracking
As discussed previously, we should design and train a deep convolutional neural network for being able to track the tongue and other facial points of the mice. For this purpose, we will use [DeepLabCut](http://www.mousemotorlab.org/deeplabcut), an incredibly powerful software toolbox which takes advantage of deep neural networks specially designed and refined for **tracking animal motion**. This toolbox had already been used by Jun for showing its potential advantages in a preliminary test: we can observe below that the neural network was able to track the exact coordinate of each point of interest during the whole video. 

<div align="middle">
  <img src="Multimedia/normal_speed.gif">
  <img width="380" height="380" src="Multimedia/plot_trajectories.jpg">
</div>

The complete procedure of my implementation of the neural network can be found on [tongue_tracking](./tongue_tracking.ipynb). The model is trained thanks to **126 videos with 16 labeled frames** each. For reducing the time of the manual labelling phase, we only consider **5 points of interest**: the snout, the paw, the jaw, the tongue and the spout. The model should be trained in a compting enviroment with CUDA installed on it for taking advantage of the parallel computing power of GPUs. 

### 3.2 Results discussion
The results are stored in a hd5 file in the same directory where the videos reside. Moreover, DeepLabCut can directly plot the results for better understading the tracking. For example, in the graph below we can see the evolution (frame by frame) of the coordinates (x-solid, y-dashed) for different tracking points. We observe that the **spout** is completely static which is a good sign as well as the **jaw** and the **paw**. We also remark that the **tongue** only appears during very little frames and that´s because for most of the experiment the animal is not licking so the tongue can´t be tracked. The **snout** also changes slightly of position while the animal is licking or even before, which could help us predict when the animal is going to lick even before it does. 

<div align="middle">
  <img src="Multimedia/plot.png">
</div>

Another perspective of the results can be analysed by **plotting the likelihood** computed by the neural network of the point being correctly tracked or not. We can see that every point has 100% probability of correct tracking except for the tongue.The probability of the tongue is 0 except when the mouse is licking so finding the frame number where the tongue probability is high could be a potential way of detecting the **exact timing of the lick**

<div align="middle">
  <img src="Multimedia/plot-likelihood.png">
</div>

<a id = 'cluster'></a>
## 4. Neural data clustering

Finally, I worked on developing some clusteting methods for **reducing the high-dimensionality** of the experimental dataset obtained from the **neural data collection**. Clustering is simply defined by separating objects into groups that share similar features with the purpose of identifying underlying patterns between the datapoints. 

In this case, I performed Principal Component Analysis (PCA) and K-means custering in order to particulary visualise the high-dimensional dataset of the neural recordings as its shown below. The principle is to cluster the datapoints by the K-means algorithm and plot them against three PCA features with highest variance contribution. The clusters should correspond with the **trial types** (false alarm, early lick...). All the details about this analysis can be found on the [data_clustering](./data_clustering.ipynb) notebook and the resulting graph is also shown interactively in [this link](https://chart-studio.plotly.com/~emfernan/3/#/plot)



<div align="middle">
  <img src="Multimedia/clusters.png">
</div>